# Code to generate Frustration fouls and Euro Fouls

In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from nba_api.stats.static import teams
from pbpstats.client import Client
from tqdm import tqdm
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

In [ ]:
from pbpstats.client import Client

settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "C:/Users/spannala/Documents/Sra_coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
season = client.Season("nba", "2020-21", "Regular Season")
games_id = []
k = 0
for final_game in season.games.final_games:
    k += 1
    clear_output(wait=True)
    print(k)
    games_id.append(final_game['game_id'])

In [ ]:
settings = {
    "Boxscore": {"source": "web", "data_provider": "data_nba"},
    "Possessions": {"source": "web", "data_provider": "data_nba"},
    "dir": "C:/Users/spannala/Documents/Sra_coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
k = 0
for gameid in tqdm(games_id):
    clear_output(wait=True)
    k += 1
    # print(k)
    try:
        games_list.append(client.Game(gameid))
    except:
        print(gameid)
        bad_games_list.append(gameid)
        continue

In [ ]:
len(bad_games_list)

In [ ]:
year = 2020
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

In [ ]:
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
pos_store = []
TO_Miss_pID = []
Foul_pID = []
Foul_tID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= 5:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                print ("Foul Player: {0}".format(possession_event.player1_id))
                ii += 1
                # print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
Player_Name = []
a = Foul_pID
Foul_pID1, Fouls = np.unique(a, return_counts=True)  

for pID in Foul_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [ ]:
data = pd.DataFrame({'Foul_pID':Foul_pID1,'Player_Name':Player_Name,'Fouls':Fouls})
data.to_csv ('Fast_Fouls_20_21_updated.csv',index=False)

In [ ]:
Team_Name = [] 
a = Foul_tID
Foul_ptD1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

In [ ]:
data = pd.DataFrame({'Foul_tID':Foul_tID1,'Team_Name':Team_Name,'Fouls':Fouls})
data.to_csv ('Fast_Fouls_20_21_teams_updated.csv',index=False)

In [ ]:
FFoul_pID =[]
k = 0
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])
        k +=1
        print(k)
        clear_output(wait=True)

In [ ]:
Player_Name = []
a = FFoul_pID
FFoul_pID, Fouls = np.unique(a, return_counts=True)
for pID in FFoul_pID:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))   

In [ ]:
data = pd.DataFrame({'FFoul_pID':FFoul_pID,'Player_Name':Player_Name,'Fouls':Fouls})
data.to_csv ('Frustration_Fouls_20_21_updated.csv',index=False)